This notebook is to create a CSV that aggregates ride information for each day

In [1]:
import pandas as pd
import duckdb

In [2]:
daily_rentals_columns = ['ride_id', 'started_at', 'ended_at', 'start_station_id', 'end_station_id', 'member_casual']

daily_rentals = pd.read_csv('daily_rent_detail.csv', usecols=daily_rentals_columns)

D:\Programs\Anaconda\Dwiggins\AppData\Local\Temp\ipykernel_15636\3272475400.py:3: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  daily_rentals = pd.read_csv('daily_rent_detail.csv', usecols=daily_rentals_columns)


In [3]:
daily_rentals['ride_id'] = daily_rentals['ride_id'].astype('string')
daily_rentals['started_at'] = pd.to_datetime(daily_rentals['started_at'], format = 'ISO8601')
daily_rentals['ended_at'] = pd.to_datetime(daily_rentals['ended_at'], format = 'ISO8601')
daily_rentals['start_station_id'] = daily_rentals['start_station_id'].astype('string')
daily_rentals['end_station_id'] = daily_rentals['end_station_id'].astype('string')
daily_rentals['member_casual'] = daily_rentals['member_casual'].astype('string')

daily_rentals.dtypes

ride_id             string[python]
started_at          datetime64[ns]
ended_at            datetime64[ns]
start_station_id    string[python]
end_station_id      string[python]
member_casual       string[python]
dtype: object

In [5]:
ride_data = duckdb.sql("""
    SELECT
        started_at::date AS date,
        COUNT(ride_id) AS total_rides,
        ROUND(SUM(EXTRACT(EPOCH FROM (ended_at - started_at)::INTERVAL)/60), 2) AS total_duration_mins
    FROM daily_rentals
    GROUP BY started_at::date
    ORDER BY date
""").fetchdf()

ride_data

,date,total_rides,total_duration_mins
0,2020-05-01,2969,91076.58
1,2020-05-02,9028,897015.15
2,2020-05-03,3663,190313.00
3,2020-05-04,3718,230021.55
4,2020-05-05,2374,90237.02
...,...,...,...
1579,2024-08-27,21795,328821.32
1580,2024-08-28,20796,292875.13
1581,2024-08-29,16217,220978.09
1582,2024-08-30,17877,266101.09


In [6]:
ride_data = ride_data.drop(ride_data[ride_data['total_duration_mins'] < 0].index, axis = 0)
ride_data = ride_data.reset_index()

In [7]:
ride_data.to_csv('daily_rides_agg.csv', index = False)